# **Neural Networks for SIR models**

# **Keras -- Practical Session 2**
## **Modelos Epidemiológicos, 2020**

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from time import time
import tqdm

from scipy.integrate import solve_ivp
from scipy.integrate import odeint

from itertools import product

from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model, model_to_dot

from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

#need to install pydot and graphviz libraries
# to do: open your terminal and access your keras environment
# execute the following command
#    conda install -c anaconda pydot graphviz

## Create synthetic data

In [ ]:
def sirN(y, t, beta, gamma, N):
    S,I,R = y
    dSdt  = -beta*I*S / N
    dIdt  = beta*I*S / N - gamma*I
    dRdt  = gamma*I
    return [dSdt, dIdt, dRdt]

In [ ]:
Nmin = 1
Nmax = 1
N = 789

h = 1
Pop = np.linspace(Nmin,Nmax,1+int((Nmax-Nmin)/h))
print(Pop)

In [ ]:
bmin = 1.5
bmax = 2.0 
hb = 0.005
beta = np.linspace(bmin, bmax, 1+int((bmax-bmin)/hb))
print(beta)

In [ ]:
gmin = 0.75
gmax = 1.25
hg = 0.005
gamma = np.linspace(gmin, gmax, 1+int((gmax-gmin)/hg))
print(gamma)

In [ ]:
x = product(beta, gamma, Pop)
#print(x)

In [ ]:
T = 25.
tspan = [0., T]
t = np.linspace(0, T, 501)

print(t)

In [ ]:
# storage
sol = []

for i in x:  
    # parameters
    bet = i[0]
    gam = i[1]
    n = i[2]

    S0     = (N-1.)/N   # n-1.
    I0     = 1./N       # 1.
    R0     = 0.
    params = (bet, gam, n)
    y0     = [S0, I0, R0]

    # solve SIR model
    #yt = solve_ivp(sir, tspan, y0, t_eval=teval, args=params)
    yt = odeint(sirN, y0, t, args=params)

    # store solution
    sol.append(np.hstack([bet, gam, n, S0, I0, R0, t.ravel(), (yt.T).ravel()]))

print(t.shape)
print(yt.shape)

In [ ]:
sol = np.array(sol).astype(np.float32)
print(sol.shape)

In [ ]:
print(sol.min(), sol.max())
#print(sol.min(), sol.max())

## Data

In [ ]:
#parametros
M = 789.
beta  = 1.89
gamma = 0.48
S0    = (M-1.)/M
I0    = 1./M
R0    = 0.
y0    = np.array([S0, I0, R0])
params = (beta, gamma, 1.)

val_sol = odeint(sirN, y0, t, args=params)
St = val_sol[:,0]
It = val_sol[:,1]
Rt = val_sol[:,2]

In [ ]:
print(St.shape, It.shape, Rt.shape)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(St, label=r'$S(t)$')
plt.plot(It, label=r'$I(t)$')
plt.plot(Rt, label=r'$R(t)$')
plt.xlabel('t')
plt.ylabel('Solution')
plt.title(r'SIR model')
plt.legend()
plt.show()

In [ ]:
# add noise to I
It_noise = st.poisson.rvs(100.*It)/100.

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(t, It, label='Real')
plt.plot(t, It_noise, 'r.', label='Observed')
plt.xlabel('t')
plt.ylabel('Infected')
plt.title(r'Infected $I(t)$, real and observed')
plt.legend()
plt.show()

### Training and Test data

In [ ]:
# set start and end interval for training data
tmin_tr = 0
tmax_tr = 161

# set start and end interval for testing data
tmin_ts = 161
tmax_ts = 401

S_train = sol[:,-1503:-1503+tmax_tr:]
I_train = sol[:,-1002:-1002+tmax_tr:]
R_train = sol[:,-501:-501+tmax_tr:]

S_test = sol[:,-1503+tmax_tr:-1503+tmax_ts]
I_test = sol[:,-1002+tmax_tr:-1002+tmax_ts]
R_test = sol[:,-501+tmax_tr:-501+tmax_ts]

# full interval (train + test)
I_full = sol[:,-1002:-1002+tmax_ts]

In [ ]:
print(S_train.shape, I_train.shape, R_train.shape)
print(S_test.shape, I_test.shape, R_test.shape)
print(I_full.shape)

In [ ]:
print(I_train.min(), I_train.max())
print(I_test.min(), I_test.max())

In [ ]:
input_shape = I_train.shape[1:]
print(input_shape)

output_shape = I_full.shape[1:]
print(output_shape)

## Train neural network

In [ ]:
def Model_1(input_shape, output_shape):
    I = Input(shape=input_shape, name='input')
    X = Dense(128, activation='relu', name='dense1')(I)
    X = Dropout(0.25, name='dropout1')(X)
    X = Dense(128, activation='relu', name='dense2')(I)
    X = Dropout(0.25, name='dropout2')(X)
    X = Dense(64, activation='relu', name='dense3')(X)
    X = Dropout(0.25, name='dropout3')(X)
    X = Dense(64, activation='relu', name='dense4')(X)
    X = Dropout(0.25, name='dropout4')(X)
    X = Dense(output_shape[0], activation=None, name='output')(X)
    
    model = Model(I, X, name='SIR-Model_1')
    return model

In [ ]:
if 'model' in globals():
    del model
    model = None
    
model = Model_1(input_shape, output_shape)

In [ ]:
model.summary()

In [ ]:
# create figure of model  (uncomment after installing pydot and graphviz)
plot_model(model, to_file='model_1.png', show_shapes=True, show_layer_names=True)

In [ ]:
# settings
alpha = 1e-4
decay = 1e-5
pat = 20         # parameter for early stopping

#define optimizer
opt = optimizers.Adam(learning_rate=alpha, decay=decay)

# compile model
model.compile(optimizer=opt, loss='mse', metrics=['mae'])

# early stopping settings
path = ''
modelname = 'SIR-Model1_01.h5'
callbacks_list = [EarlyStopping(monitor='val_loss', patience=pat),
                  ModelCheckpoint(filepath=path+modelname, monitor='val_loss', save_best_only=True)]

In [ ]:
# training model
history = model.fit(x=I_train, y=I_full, epochs=200, batch_size=64, callbacks=callbacks_list, validation_split=0.16666,
                    shuffle=True, verbose=1)

#history = model.fit(x=I_train, y=I_full, epochs=200, batch_size=64, validation_split=0.16666, shuffle=True)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
# plot training history

loss = history.history['loss']
val_loss = history.history['val_loss']
metr = history.history['mae']
val_metr = history.history['val_mae']

start = 20
epochs = range(start, len(loss))

# figure
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss[start:], 'b', label='Training loss')
plt.plot(epochs, val_loss[start:], 'g', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, metr[start:], 'r', label='Training mae')
plt.plot(epochs, val_metr[start:], 'g', label='Validation mae')
plt.title('Training and validation metrics')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

## Prediction

In [ ]:
# downloading best saved model

if 'model' in globals():
    del model
    model = None
    
model = load_model('SIR-Model1_01.h5')

In [ ]:
Y_test = It_noise[:tmax_tr].reshape(1,-1)
print(Y_test.shape)

prediction = model.predict(Y_test)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(t[:tmax_tr], It_noise[:tmax_tr], 'b.', label='Observed data')
plt.plot(t[:tmax_tr], It[:tmax_tr], 'g-', label='Real data')
plt.plot(t[tmax_tr:tmax_ts], It_noise[tmax_tr:tmax_ts], 'r.', label='Expected')
plt.plot(t[:tmax_ts], prediction.ravel(), 'k-', label='Prediction')
plt.xlabel('t')
plt.ylabel('Infected')
plt.title(r'Infected $I(t)$, real and observed')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(t[:tmax_tr], It_noise[:tmax_tr], 'r.', label='Expected')
plt.plot(t[:tmax_tr], prediction.ravel()[:tmax_tr], 'k-', label='Prediction')
plt.xlabel('t')
plt.ylabel('Infected')
plt.title(r'Infected $I(t)$, real and observed')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(t[tmax_tr:tmax_ts], It_noise[tmax_tr:tmax_ts], 'r.', label='Expected')
plt.plot(t[tmax_tr:tmax_ts], prediction.ravel()[tmax_tr:tmax_ts], 'k-', label='Prediction')
plt.xlabel('t')
plt.ylabel('Infected')
plt.title(r'Infected $I(t)$, real and observed')
plt.legend()
plt.show()

## Training a second model (to predict S, I, R)

### Training and Test data

In [ ]:
tmin_tr = 0
tmax_tr = 161

tmin_ts = 161
tmax_ts = 401

X_tr = sol[:,-1002:-1002+tmax_tr:]

# our Y data is a concatenation of the S(t), I(t) and R(t) information
Y_tr = np.hstack([sol[:,-1503:-1503+tmax_ts:], sol[:,-1002:-1002+tmax_ts:], sol[:,-501:-501+tmax_ts:]])

In [ ]:
print(X_tr.shape, Y_tr.shape)

In [ ]:
print(X_tr.min(), X_tr.max())
print(Y_tr.min(), Y_tr.max())

In [ ]:
input_shape = X_tr.shape[1:]
print(input_shape)

output_shape =Y_tr.shape[1:]
print(output_shape)

## Train neural network

In [ ]:
def Model_2(input_shape, output_shape):
    I = Input(shape=input_shape, name='input')
    X = Dense(256, activation='relu', name='dense0')(I)
    X = Dropout(0.25, name='dropout0')(X)
    X = Dense(256, activation='relu', name='dense1')(I)
    X = Dropout(0.25, name='dropout1')(X)
    X = Dense(128, activation='relu', name='dense2')(X)
    X = Dropout(0.25, name='dropout2')(X)
    X = Dense(128, activation='relu', name='dense3')(X)
    X = Dropout(0.25, name='dropout3')(X)
    X = Dense(64, activation='relu', name='dense4')(X)
    X = Dropout(0.25, name='dropout4')(X)
    X = Dense(output_shape[0], activation='sigmoid', name='output')(X)
    
    model = Model(I, X, name='SIR-Model_2')
    return model

In [ ]:
if 'model' in globals():
    del model
    model = None
    
model = Model_2(input_shape, output_shape)

In [ ]:
model.summary()

In [ ]:
# create figure of model  (uncomment after installing pydot and graphviz)
plot_model(model, to_file='model_2.png', show_shapes=True, show_layer_names=True)

<img src='model_2.png'/>

In [ ]:
# settings
alpha = 1e-4
decay = 1e-6
pat = 20

# define optimizer
opt = optimizers.Adam(learning_rate=alpha, decay=decay)

# compile model
model.compile(optimizer=opt, loss='mae', metrics=['mse'])

# early stopping settings
path = ''
modelname = 'SIR-Model2_01.h5'
callbacks_list = [EarlyStopping(monitor = 'val_loss', patience=pat),
                  ModelCheckpoint(filepath=path+modelname, monitor='val_loss', save_best_only=True)]

In [ ]:
# training model
history = model.fit(x=X_tr, y=Y_tr, epochs=400, batch_size=64, callbacks=callbacks_list, validation_split=0.16666,
                    shuffle=True, verbose=1)

#history = model.fit(x=I_train, y=I_full, epochs=200, batch_size=64, validation_split=0.16666, shuffle=True)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
keys = list(history_dict.keys())

In [ ]:
# plot training history

loss = history.history[keys[0]]
metr = history.history[keys[1]]
val_loss = history.history[keys[2]]
val_metr = history.history[keys[3]]

start = 30
epochs = range(start, len(loss))

# figure
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss[start:], 'b', label='Training loss')
plt.plot(epochs, val_loss[start:], 'g', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, metr[start:], 'r', label='Training mse')
plt.plot(epochs, val_metr[start:], 'g', label='Validation mse')
plt.title('Training and validation metrics')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

## Prediction

In [ ]:
# downloading best saved model

if 'model' in globals():
    del model
    model = None
    
model = load_model('SIR-Model2_01.h5')

In [ ]:
#Y_test = It_noise[:tmax_tr].reshape(1,-1)
Y_test = It[:tmax_tr].reshape(1,-1)
print(Y_test.shape)

prediction = model.predict(Y_test)
print(prediction.shape)

In [ ]:
def moving_average(x, n=3) :
    ret = np.cumsum(x)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n-1:] / n

In [ ]:
ww = 21

S_ma = moving_average(prediction[0,0:1*tmax_ts], n=ww)
I_ma = moving_average(prediction[0,1*tmax_ts:2*tmax_ts], n=ww)
R_ma = moving_average(prediction[0,2*tmax_ts:3*tmax_ts], n=ww)

In [ ]:
labels = [r'$S(t)$', r'$I(t)$', r'$R(t)$']
titles = ['Susceptible', 'Infected', 'Removed']
plt.figure(figsize=(18,4))
for i in range(0,3):
    plt.subplot(1,3,i+1)
    if i == 1:
        plt.plot(t[:tmax_tr], It_noise[:tmax_tr], 'b.', label='Observed data')
        #plt.plot(t[tmax_tr:tmax_ts], It_noise[tmax_tr:tmax_ts], 'r.', label='Expected')
    if i == 0:
        plt.plot(t[:tmax_tr], St[:tmax_tr], 'g-', label='Real data')
        plt.plot(t[tmax_tr:tmax_ts], St[tmax_tr:tmax_ts], 'r-', label='Expected')
        plt.plot(t[ww-1:tmax_ts], S_ma, 'k-', label='Prediction')
    elif i == 1:
        plt.plot(t[:tmax_tr], It[:tmax_tr], 'g-', label='Real data')
        plt.plot(t[tmax_tr:tmax_ts], It[tmax_tr:tmax_ts], 'r-', label='Expected')
        plt.plot(t[ww-1:tmax_ts], I_ma, 'k-', label='Prediction')
    elif i == 2:
        plt.plot(t[:tmax_tr], Rt[:tmax_tr], 'g-', label='Real data')
        plt.plot(t[tmax_tr:tmax_ts], Rt[tmax_tr:tmax_ts], 'r-', label='Expected')
        plt.plot(t[ww-1:tmax_ts], R_ma, 'k-', label='Prediction')
        
    #plt.plot(t[:tmax_ts], prediction.ravel()[i*tmax_ts:(i+1)*tmax_ts], 'k-', label='Prediction')
    plt.xlabel('t')
    plt.ylabel(labels[i])
    plt.title(titles[i] + ' ' + labels[i] + ', real and observed')
    plt.legend()
plt.show()